# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../output_data/cities.csv")
weather_df.head(5)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_df["Humidity"]



In [4]:
# Plot Heatmap
fig_map = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_int = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig_map.add_layer(heat_layer)

# Display figure
fig_map

Figure(layout=FigureLayout(height='420px'))

In [5]:
max_int

290

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:

narrow_city = weather_df.loc[(weather_df["Wind Speed"] <= 5) & (weather_df["Cloudiness"] == 0) & \
                        (weather_df["Max Temp"] >= 65) & (weather_df["Max Temp"] <= 85)].dropna()

narrow_city

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
346,346,chiredzi,0,ZW,1558378856,42,-21.05,31.67,65.76,0.78
538,538,la seyne-sur-mer,0,FR,1558378897,72,43.10,5.88,69.01,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:

hotel_df = narrow_city.loc[:,["City","Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
125,arraial do cabo,BR,-22.97,-42.02,
139,vaini,IN,15.34,74.49,
297,mogok,MM,22.92,96.51,
346,chiredzi,ZW,-21.05,31.67,
538,la seyne-sur-mer,FR,43.10,5.88,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
   
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    
    params["location"] = f"{lat},{lng}"

    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
  
    results = response['results']
    
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    

print("-------End of Search-------")

Retrieving Results for Index 125: arraial do cabo.
Missing field/result... skipping.
------------
Retrieving Results for Index 139: vaini.
Missing field/result... skipping.
------------
Retrieving Results for Index 297: mogok.
Missing field/result... skipping.
------------
Retrieving Results for Index 346: chiredzi.
Missing field/result... skipping.
------------
Retrieving Results for Index 538: la seyne-sur-mer.
Missing field/result... skipping.
------------
-------End of Search-------


In [10]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
125,arraial do cabo,BR,-22.97,-42.02,
139,vaini,IN,15.34,74.49,
297,mogok,MM,22.92,96.51,
346,chiredzi,ZW,-21.05,31.67,
538,la seyne-sur-mer,FR,43.10,5.88,


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig_map.add_layer(markers)

# Display Map

fig_map

Figure(layout=FigureLayout(height='420px'))